# Isochrones for public parking lots in Tel Aviv

Created by [Dan Fishgold](https://dan.city)

Based on [Elad's LRT network isochrones](https://github.com/elad661/metroTLV_walkshed)

In [6]:
# Basic setup
import json
import pandas as pd
from tqdm.notebook import tqdm
import geopandas as gpd
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
import shapely.geometry
import sklearn
import re
from fetch_statuses import get_lot_names

ox.settings.log_console = True

In [2]:
# configure basic parameters
 # (Download the entire metropolitian area, and then some. Too bad OSM doesn't have a relationship for the Tel Aviv Metropolitan Area)
place = ["Tel Aviv District, Israel", "Center District, Israel"]
network_type = "walk"
trip_times = [5, 7, 10]  # in minutes
travel_speed = 4.5  # very approximate walking speed in km/hour (real humans might walk slower or faster)

## Download and prep the street network

In [11]:
# download the street network
graph = ox.graph_from_place(place, network_type=network_type)

In [12]:
# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for _, _, _, data in graph.edges(data=True, keys=True):
    data["time"] = data["length"] / meters_per_minute

In [13]:
projected_graph = ox.project_graph(graph)
crs = projected_graph.graph['crs']

## Load night lots

From [Tel Aviv's GIS server](https://gisn.tel-aviv.gov.il/arcgis/rest/services/IView2/MapServer/488)

In [15]:
night_lots = gpd.read_file('https://gisn.tel-aviv.gov.il/arcgis/rest/services/IView2/MapServer/488/query?where=1%3D1&outFields=*&f=json').to_crs(crs).set_index('oid')

In [16]:
night_lots = night_lots[(night_lots.ahuzat_hof_code != 0) & (~night_lots.ahuzat_hof_code.isna())]
night_lots['ahuzat_hof_code'] = night_lots['ahuzat_hof_code'].astype(int)

## Load Ahuzot Hahof lots

In [14]:
try:
    gdf = gpd.read_file("https://www.ahuzot.co.il/map/ParkingMap.aspx?gx=1234", driver='KML').to_crs(crs)
except:
    gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
    gdf = gpd.read_file("https://www.ahuzot.co.il/map/ParkingMap.aspx?gx=1234", driver='KML').to_crs(crs)

gdf['id'] = gdf.apply(lambda lot: int(re.search(r'href="https://www.ahuzot.co.il/Parking/ParkingDetails/\?ID=(\d+)"', lot.Description).group(1)), axis=1)
gdf = gdf.set_index('id')
del gdf['Description']

In [31]:
names = get_lot_names()
names_with_int_index = {int(id): name for (id, name) in names.items()}
gdf['name'] = pd.Series(names_with_int_index)
del gdf['Name']

In [18]:
gdf['nearest_node'] = gdf.apply(lambda lot: ox.distance.nearest_nodes(projected_graph, lot.geometry.x, lot.geometry.y), axis=1)

In [46]:
pd.set_option('display.max_rows', 500)
night_lots.sjoin_nearest(gdf, distance_col='distance', how='left', max_distance=100).reindex(['shem', 'name', 'distance', 'geometry'], axis='columns').sort_values(by='distance', ascending=False)


,shem,name,distance,geometry
oid,,,,
1,בזל,חניון בזל,25.260166,POINT (667959.727 3551775.939)
10,חנה וסע ארלוזורוב,חניון ארלוזורוב - חנה וסע,7.665766,POINT (669534.806 3551303.136)
20,התרבות,חניון התרבות,6.776358,POINT (668011.171 3549802.814)
21,"פלמ""ח","חניון פלמ""ח",6.270700,POINT (669198.694 3548802.327)
46,הרב קוק,חניון הרב קוק,6.169624,POINT (666871.286 3549639.662)
14,סינרמה,חניון סינרמה,6.078150,POINT (669091.528 3549000.154)
22,לולאה,חניון לולאה,6.067664,POINT (669862.015 3550931.534)
15,"בית האצ""ל - החוף המערבי","חניון בית האצ""ל",5.679445,POINT (666089.478 3548362.421)
39,בית הדר א,חניון בית הדר,5.592612,POINT (667742.949 3548888.980)


## Generate the isochrones

In [ ]:
# This function makes the isochrones, will be reused later
def make_iso_polys(G, center_node, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = {}
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")

        node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index("id")

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union

        # try to fill in surrounded areas so shapes will appear solid and
        # blocks without white space inside them
        if infill and hasattr(new_iso, 'exterior'):
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys[trip_time] = new_iso
    return isochrone_polys

def get_geojson_geometry(polygon):
    """Get geojson-compatible geometry, projected to a useful CRS"""
    geometry = ox.projection.project_geometry(polygon, crs=projected_graph.graph['crs'], to_latlong=True)[0]
    rounded_geometry = shapely.wkt.loads(shapely.wkt.dumps(geometry, rounding_precision=7))
    return shapely.geometry.mapping(rounded_geometry)



In [ ]:
time_polys = {time: [] for time in trip_times}
for _, lot in tqdm(gdf.iterrows()):
    polys = make_iso_polys(projected_graph, lot['nearest_node'], edge_buff=25, node_buff=0, infill=True)
    for time, polygon in polys.items():
        properties = lot.to_dict()
        del properties['geometry']
        time_polys[time].append(dict(poly=polygon.simplify(1), properties=properties))

## Save the isochrones

In [ ]:
for time, polys in time_polys.items():
    features = []
    for polygon in polys:
        properties = {'time': time, **polygon['properties']}
        geometry = get_geojson_geometry(polygon['poly'])
        features.append(dict(type='Feature', properties=properties, geometry=geometry))

    # Save geojson
    geojson = { "type": "FeatureCollection", "features": features }
    with open(f'./parking_lot_isochrones_{time}min.geojson', 'w') as f:
        json.dump(geojson, f)